In [2]:
# Import necessary packages
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones and output
iso_dir = 'isochrones/'
output_dir = 'output_diagrams/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Estimation variables
star_index = 0      # Set which star in the CSV to analyze
num_top_predictions = 10  # Control how many top predictions to plot

# AKs values from 0 to 20 in increments of 2
extinction_values = np.arange(0, 22, 2)

# Define static isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F162M', 'jwst,F182M']
filters = ['m_jwst_F162M', 'm_jwst_F182M']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Define age array
log_age_arr = np.log10(level_ages)

# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../../../s284-162-182.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines])

# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    for i, iso in enumerate(iso_grid):
        for star in iso.points:
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            results.append([chi_square, star['mass'], 10 ** log_age_arr[i]])
    
    return sorted(results, key=lambda x: x[0])

# Ensure both isochrone and output directories exist before the loop
os.makedirs(iso_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


# Loop through each extinction value, delete old isochrones, and generate diagrams
for AKs in extinction_values:
    # Clear previous isochrones
    for file in os.listdir(iso_dir):
        os.remove(os.path.join(iso_dir, file))
    
    # Generate isochrone grid
    instances = np.array([
        synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
        for log_age in log_age_arr
    ])

    # Run chi-square minimization and get top results
    sorted_results = chi_square_reverse_model(instances, sample_mags[star_index])
    top_results = sorted_results[:num_top_predictions]

    # Create figure with two subplots side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    cmap = plt.get_cmap('coolwarm')

    # Color-magnitude diagram (CMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax1.plot(instance.points[filters[0]] - instance.points[filters[1]], 
                 instance.points[filters[1]], color=color)
    ax1.set_xlabel('F162M - F182M')
    ax1.set_ylabel('F182M')
    ax1.invert_yaxis()
    ax1.grid(True)  # Add grid lines to CMD

    # Magnitude-magnitude diagram (MMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax2.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)
    ax2.set_xlabel('F162M')
    ax2.set_ylabel('F182M')
    ax2.invert_xaxis()
    ax2.invert_yaxis()
    ax2.grid(True)  # Add grid lines to MMD

    # Create colorbar for both diagrams
    norm = Normalize(vmin=min(level_ages), vmax=max(level_ages))
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=[ax1, ax2])
    cbar.set_label('Age (millions of years)')
    cbar.set_ticks(level_ages)
    cbar.set_ticklabels([f'{age/1e6:.1f}' for age in level_ages])

    # Plot top results on CMD with color gradient and labels
    for j, (chi_square, mass, age) in enumerate(top_results):
        closest_age_idx = np.argmin(abs(level_ages - age))
        iso_instance = instances[closest_age_idx]
        mass_idx = np.argmin(abs(iso_instance.points['mass'] - mass))
        color = cmap(j / (num_top_predictions - 1))
        ax1.plot(iso_instance.points[mass_idx][filters[0]] - iso_instance.points[mass_idx][filters[1]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")
        ax2.plot(iso_instance.points[mass_idx][filters[0]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")

    # Plot reference star in both diagrams
    ref_star_color = 'gold'  # Optional: distinguish reference star color
    ref_star_marker = '*'  # Optional: distinguish reference star marker

    ax1.plot(sample_mags[star_index][0] - sample_mags[star_index][1], sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    ax2.plot(sample_mags[star_index][0], 
         sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    # Legends and saving
    ax1.legend()
    ax2.legend()
    fig.suptitle(f"AKs = {AKs}")
    plt.savefig(os.path.join(output_dir, f'CMD_MMD_AKs_{AKs}.png'))
    plt.close()

Isochrone generation took 2.530566 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:57:57.715938   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 21.99
Starting filter: jwst,F182M   Elapsed time: 1.75 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 22.18
      Time taken: 3.49 seconds
Isochrone generation took 2.422412 s.
Making photometry for isochrone: log(t) = 6.18  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:58:03.659588   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F162M = 22.36
Starting filter: jwst,F182M   Elapsed time: 1.77 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F182M = 22.56
      Time taken: 3.51 seconds
Isochrone genera

      Time taken: 3.48 seconds
Changing to logg=5.00 for T=  1631 logg=4.04
Isochrone generation took 2.289867 s.
Making photometry for isochrone: log(t) = 6.95  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:59:30.517675   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F162M = 25.00
Starting filter: jwst,F182M   Elapsed time: 1.75 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F182M = 24.84
      Time taken: 3.49 seconds
Changing to logg=5.00 for T=  1602 logg=4.04
Isochrone generation took 2.311969 s.
Making photometry for isochrone: log(t) = 6.98  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:59:36.355439   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F162M = 25.15
Starting filter: jwst,F182M   Elapsed time: 1.75 seconds
Starting

      Time taken: 3.47 seconds
Changing to logg=5.00 for T=  1773 logg=4.01
Isochrone generation took 2.419528 s.
Making photometry for isochrone: log(t) = 6.88  AKs = 2.00  dist = 4500
     Starting at:  2024-10-29 04:01:06.908995   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F162M = 27.85
Starting filter: jwst,F182M   Elapsed time: 1.74 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F182M = 27.14
      Time taken: 3.48 seconds
Changing to logg=5.00 for T=  1711 logg=4.02
Isochrone generation took 2.336182 s.
Making photometry for isochrone: log(t) = 6.90  AKs = 2.00  dist = 4500
     Starting at:  2024-10-29 04:01:12.754510   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F162M = 28.16
Starting filter: jwst,F182M   Elapsed time: 1.74 seconds
Starting

      Time taken: 2.38 seconds
Changing to logg=5.00 for T=  1870 logg=3.98
Isochrone generation took 1.734433 s.
Making photometry for isochrone: log(t) = 6.78  AKs = 4.00  dist = 4500
     Starting at:  2024-10-29 04:02:33.975590   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1870 K  m_jwst_F162M = 30.83
Starting filter: jwst,F182M   Elapsed time: 1.20 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1870 K  m_jwst_F182M = 29.60
      Time taken: 2.41 seconds
Changing to logg=5.00 for T=  1838 logg=3.99
Isochrone generation took 1.732188 s.
Making photometry for isochrone: log(t) = 6.81  AKs = 4.00  dist = 4500
     Starting at:  2024-10-29 04:02:38.139329   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1838 K  m_jwst_F162M = 30.99
Starting filter: jwst,F182M   Elapsed time: 1.20 seconds
Starting

      Time taken: 2.91 seconds
Isochrone generation took 2.306311 s.
Making photometry for isochrone: log(t) = 6.65  AKs = 6.00  dist = 4500
     Starting at:  2024-10-29 04:03:44.221416   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F162M = 33.95
Starting filter: jwst,F182M   Elapsed time: 1.45 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F182M = 31.90
      Time taken: 2.88 seconds
Changing to logg=5.00 for T=  1960 logg=3.95
Isochrone generation took 2.295037 s.
Making photometry for isochrone: log(t) = 6.70  AKs = 6.00  dist = 4500
     Starting at:  2024-10-29 04:03:49.429963   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F162M = 34.03
Starting filter: jwst,F182M   Elapsed time: 1.45 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  

...

KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f0964dffa60> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f07ae5e7ae8> (for post_execute):


KeyboardInterrupt: 